# Data to Dome: Visualizing Gamma Ray Bursts in WWT

For this month’s tutorial we will create a visualization of Gamma Ray Bursts. The tutorial was created for A KICP short course for museum and planetarium staff about the Evolving Universe (http://kicp-courses.uchicago.edu/2014/index.php) held this September. Here we will only visualize the data in WWT. While similar visualizations are possible in other planetarium software packages, the process of creating them is somewhat awkward. The time domain is the next great frontier for astronomy, I encourage software vendors to follow along with the tutorial and think about how they can streamline the process of creating a similar visualization using their software.<br><br>
Mark Subbarao (msubbarao at adlerplanetarium.org)

##### Python Setup

In [1]:
from astropy.table import Table,Column
from astropy.time import Time
from astropy import units
from astropy.coordinates import SkyCoord
from astroquery.vizier import Vizier

In [2]:
#Create Vizier object, turn off default row limit
v = Vizier()
v.ROW_LIMIT = -1

##### WWT Setup

In [ ]:
from pywwt.mods import *

In [ ]:
#Connect to WWT
wwt = WWTClient() #Can pass a IP address here if WWT is running on a remote machine
wwt.new_layer_group("Sky","Dynamic Universe")

### Gamma Ray Bursts

For out data catalog we'll choose The second Fermi/GBM GRB catalog (4yr) (von Kienlin+, 2014)
Vizier catalog: J/ApJS/211/13/GBM
Which contains Fermi events from July 2007 to July 2012

In [3]:
Cats = v.get_catalogs('J/ApJS/211/13/GBM')

In [5]:
Cats[0]

_RAJ2000,_DEJ2000,TID,GRB,f_GRB,ObsTime,RAJ2000,DEJ2000,Err,Loc,Alg,Time,Erange,OtherDet,_2yr,LC,SimbadName,Rem,Det,T90,e_T90,T90st,T50,e_T50,T50st,Fl.w,e_Fl.w,PF1.w,e_PF1.w,PF2.w,e_PF2.w,PF3.w,e_PF3.w,Fl.n,e_Fl.n,PF1.n,e_PF1.n,PF2.n,e_PF2.n,PF3.n,e_PF3.n
deg,deg,,,,"""h:m:s""",deg,deg,deg,,,ms,keV,,,,,,,s,s,s,s,s,s,mJ / m2,mJ / m2,ph s / cm2,ph s / cm2,ph s / cm2,ph s / cm2,ph s / cm2,ph s / cm2,mJ / m2,mJ / m2,ph s / cm2,ph s / cm2,ph s / cm2,ph s / cm2,ph s / cm2,ph s / cm2
float64,float64,int32,bytes7,bytes3,bytes13,float32,float32,float32,bytes9,int16,int16,bytes6,bytes28,int16,bytes2,bytes10,bytes1,bytes13,float32,float32,float64,float32,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
41.9000,8.5000,80714086,080714B,,02:04:12.0534,41.90,8.50,7.5,Fermi-GBM,10,512,47-291,K,1,LC,GRB080714B,,3+4+8,5.376,2.360,-0.768,2.816,0.810,-0.256,6.8e-07,4.1e-08,3.82,1.06,2.24,0.36,1.54,0.18,3.5e-07,1.7e-08,1.52,0.74,0.91,0.36,0.43,0.18
187.5000,-74.0000,80714425,080714C,,10:12:01.8376,187.50,-74.00,8.7,Fermi-GBM,17,4096,47-291,,1,LC,GRB080714C,,0+9+10,40.192,1.145,-4.352,11.776,1.619,-1.280,1.8e-06,2.1e-08,4.00,1.45,2.96,0.46,2.02,0.21,9.8e-07,1.4e-08,1.03,0.45,0.71,0.19,0.46,0.08
188.1000,-60.2000,80714745,080714A,,17:52:54.0234,188.10,-60.20,--,Swift,13,1024,47-291,"K, R, IA, S, Me, A",1,LC,GRB080714A,,5,59.649,11.276,-0.512,25.088,7.940,2.560,6.3e-06,1.4e-07,8.89,1.61,7.78,0.83,6.93,0.39,3.3e-06,6e-08,4.41,1.66,3.27,0.71,2.82,0.36
214.7000,9.9000,80715950,080715A,i,22:48:40.1634,214.70,9.90,2.0,Fermi-GBM,29,256,23-47,"K, Me, A",1,LC,GRB080715A,,0+1+2+9+10,7.872,0.272,0.128,6.144,0.264,1.088,5e-06,7.9e-08,19.42,0.95,13.58,0.45,9.91,0.22,2.5e-06,3.5e-08,10.70,0.95,6.61,0.45,3.83,0.22
147.3000,-70.0000,80717543,080717A,,13:02:35.2207,147.30,-70.00,4.7,Fermi-GBM,17,4096,47-291,,1,LC,GRB080717A,,2+10,36.609,2.985,-5.376,13.056,0.810,1.024,4.5e-06,7.7e-08,6.24,1.08,3.43,0.49,2.89,0.23,2.4e-06,4.5e-08,2.14,1.03,1.30,0.47,1.05,0.23
153.2000,-61.3000,80719529,080719A,,12:41:40.9578,153.20,-61.30,13.8,Fermi-GBM,16,4096,47-291,"K, A",1,LC,GRB080719A,,6+7+9,16.128,17.887,-4.352,8.448,1.280,-2.048,7.7e-07,2.9e-08,2.77,0.83,1.77,0.29,1.12,0.16,3.9e-07,1.5e-08,0.59,0.18,0.32,0.08,0.23,0.04
98.5000,-43.9000,80720316,080720A,,07:35:35.5476,98.50,-43.90,4.8,Fermi-GBM,19,8192,47-291,,1,LC,GRB080720A,a,,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
176.8000,-60.2000,80723557,080723B,i,13:22:21.3751,176.80,-60.20,--,Swift,8,256,47-291,"K, IA, IS, Me, A",1,LC,GRB080723B,,4,58.369,1.985,2.368,40.513,0.231,14.208,7.2e-05,2.5e-07,40.97,2.24,38.24,1.09,30.45,0.49,3.9e-05,1.1e-07,21.19,1.79,19.81,1.09,15.14,0.48


In [6]:
grbCat=Cats[0]
grbCat.keep_columns(["GRB","RAJ2000","DEJ2000","Time","ObsTime","Fl.w","Fl.n"])
grbCat.rename_column('RAJ2000', 'RA')
grbCat.rename_column('DEJ2000', 'dec')

In [7]:
grbCat

GRB,ObsTime,RA,dec,Time,Fl.w,Fl.n
,"""h:m:s""",deg,deg,ms,mJ / m2,mJ / m2
bytes7,bytes13,float32,float32,int16,float32,float32
080714B,02:04:12.0534,41.90,8.50,512,6.8e-07,3.5e-07
080714C,10:12:01.8376,187.50,-74.00,4096,1.8e-06,9.8e-07
080714A,17:52:54.0234,188.10,-60.20,1024,6.3e-06,3.3e-06
080715A,22:48:40.1634,214.70,9.90,256,5e-06,2.5e-06
080717A,13:02:35.2207,147.30,-70.00,4096,4.5e-06,2.4e-06
080719A,12:41:40.9578,153.20,-61.30,4096,7.7e-07,3.9e-07
080720A,07:35:35.5476,98.50,-43.90,8192,--,--
080723B,13:22:21.3751,176.80,-60.20,256,7.2e-05,3.9e-05


In [ ]:
#Plot the Catalog
%config InlineBackend.rc = {}
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Plot the Catalog
coordsCol=SkyCoord(grbCat['RA'],grbCat['dec'],'icrs')
fig = plt.figure (figsize=(8,6))
ax = fig.add_subplot(111,projection="mollweide", axisbg='white')
ax.grid(True)
ax.get_xaxis().tick_bottom()
ax.scatter(coordsCol.galactic.l.wrap_at(180.*units.degree).radian,\
           coordsCol.galactic.b.radian,s=6,lw=0)

Extracting the event time from this table is tricky. The time of day is in the ObsTime column, but the date is embedded in the GRB name. We'll extract the date from the GRB name and combine that with ObsTime to make a astropy time object. 

In [9]:
timeList=[]
grbList=grbCat['GRB']
for i in range(len(grbList)):
    timeString= grbList[i][2:4].decode()+'/'+grbList[i][4:6].decode()+'/'+'20'+grbList[i][0:2].decode()+' '+grbCat['ObsTime'][i].decode()
    timeList.append(timeString)

##### Export Catalog to WWT

WWT contains its own time format, unfortunately one that astropy cannot write, so we'll have to create our own custom string.

In [10]:
grbList=grbCat['GRB']
timeList=[]
for i in range(len(grbList)):
    timeString= grbList[i][2:4].decode()+'/'+grbList[i][4:6].decode()+'/'+'20'+grbList[i][0:2].decode()+' '+grbCat['ObsTime'][i].decode()
    timeList.append(timeString)
grbCat.add_column(Column(timeList,name='TimeAndDate'))
grbCat

GRB,ObsTime,RA,dec,Time,Fl.w,Fl.n,TimeAndDate
,"""h:m:s""",deg,deg,ms,mJ / m2,mJ / m2,
bytes7,bytes13,float32,float32,int16,float32,float32,str24
080714B,02:04:12.0534,41.90,8.50,512,6.8e-07,3.5e-07,07/14/2008 02:04:12.0534
080714C,10:12:01.8376,187.50,-74.00,4096,1.8e-06,9.8e-07,07/14/2008 10:12:01.8376
080714A,17:52:54.0234,188.10,-60.20,1024,6.3e-06,3.3e-06,07/14/2008 17:52:54.0234
080715A,22:48:40.1634,214.70,9.90,256,5e-06,2.5e-06,07/15/2008 22:48:40.1634
080717A,13:02:35.2207,147.30,-70.00,4096,4.5e-06,2.4e-06,07/17/2008 13:02:35.2207
080719A,12:41:40.9578,153.20,-61.30,4096,7.7e-07,3.9e-07,07/19/2008 12:41:40.9578
080720A,07:35:35.5476,98.50,-43.90,8192,--,--,07/20/2008 07:35:35.5476
080723B,13:22:21.3751,176.80,-60.20,256,7.2e-05,3.9e-05,07/23/2008 13:22:21.3751


In [11]:
#Set up WWT layer
grb_layer = wwt.new_layer("Dynamic Universe", "Gamma Ray Bursts", grbCat.colnames)
#Set visualization parameters in WWT
props_dict = {"CoordinatesType":"Spherical",\
              "MarkerScale":"Screen",\
              "PointScaleType":"Constant",\
              "ScaleFactor":"64",\
              "ShowFarSide":"True",\
              "RaUnits":"Degrees",\
              "PlotType":"Gaussian",\
              "ColorValue":"ARGBColor:255:255:255:255",\
              "TimeSeries":"False"}
grb_layer.set_properties(props_dict)
#Send data to WWT client
grb_layer.update(data=grbCat, purge_all=True, no_purge=False, show=True)

NameError: name 'wwt' is not defined

Now inside WWT we can choose how we visualize the data, we can show all the data at once or playback the events as they happen watching the GRB’s go off like popcorn across the sky. 

<img src=https://raw.githubusercontent.com/IPSScienceVisualization/python-tutorials/master/images/GRBWWT.png>